In [234]:
import jqdatasdk
import pandas as pd
from datetime import datetime,timedelta
jqdatasdk.auth("18523900898","189Qwert")
# df = jqdatasdk.get_price('600519.XSHG',count=10,end_date = '2020-08-01',frequency='daily')


In [235]:
# 获取一段时间的交易日
def get_trade_days_daily(start,end):
    '''
    获取每月月底的交易日的日历
    注意: 结束日期必须是一个月的最后一天
    :param start:
    :param end:
    :return:  list,每月最后一个交易日的列表,里面元素是datetime.date类型
    '''
    # 日频率的交易日历: Index
    index = jqdatasdk.get_trade_days(start_date=start, end_date=end)
    # 转成一个df , 索引和值都是交易日(天)
    df = pd.DataFrame(index, index=index)
    # 将Index转成DatetimeIndex
    df.index = pd.to_datetime(df.index)
    # 按月重采样,缺失值用上一个填充,那么刚好值就是想要的每月最后一个交易日
#     index_m = list(df.resample('m', how='last').iloc[:,0])
    return df.index
date = get_trade_days_daily('2019-03-01','2019-05-01')


In [237]:
for i in date[0:2]:
    # 获取每一日所有的股票
#     print(i)
#     print(i-timedelta(days=65))#
    all = jqdatasdk.get_all_securities(date=i)
    # 上市超过60日
#     print(all)
    # 过滤ST及其他具有退市标签的股票 上市超过60日
    for e in all.index:
        if 'ST' in all.loc[e,'display_name'] or '*' in all.loc[e,'display_name'] or '退' in all.loc[e,'display_name'] or i.__gt__(all.loc[e,'start_date']+timedelta(days=65))==False:
            all = all.drop(e,axis=0)
    # 过滤停牌股票
    all = jqdatasdk.get_price(all.index.tolist(),end_date=i, count=1,fields=['open', 'close', 'low', 'high','high_limit','low_limit','paused',],panel=False)
    all = all[all.paused==0]
#     print(len(all))
    # 过滤一字跌停
    all = all[all['close']-all['low_limit']>0.1]
    print(len(all))
    print(all)
    #60日最低
    b = []
    price = jqdatasdk.get_price(all['code'].tolist(), count = 60, end_date=i, frequency='daily', fields=['close'])
    print(price.info())
#     for o in all['code'].tolist():
# #         print(o)
#         price = jqdatasdk.get_price(o, count = 60, end_date=i, frequency='daily', fields=['close'])
# #         print(price)
# #         print(price.min()[0],price.iloc[-1]['close'])
#         if price.min()[0]==price.iloc[-1]['close']:
#             b.append(o)
#     print(len(b))
#     print(b)
#     # 60日最低
#     if len(b)>0:
#         stock = jqdatasdk.get_price(b,end_date=i, count=1,fields=['open', 'close', 'low', 'high'],panel=False)
#     print(stock)
    break
    
    # 去除st,含有退市标签的股票
    

Exception: 您当天的查询条数超过了每日最大查询限制：100万条；付费可增加流量权限，详情请咨询管理员，微信号：JQData02

In [ ]:
all = jqdatasdk.get_price(all.index.tolist(),end_date=date[0], count=1,fields=['open', 'close', 'low', 'high','high_limit','low_limit','paused',],panel=False)

In [ ]:
# 过滤停牌股票
def filter_paused_stock(stock_list):
	current_data = jqdatasdk.get_current_data()
	return [stock for stock in stock_list if not current_data[stock].paused]

# 过滤ST及其他具有退市标签的股票
def filter_st_stock(stock_list):
	current_data = jqdatasdk.get_current_data()
	return [stock for stock in stock_list
			if not current_data[stock].is_st
			and 'ST' not in current_data[stock].name
			and '*' not in current_data[stock].name
			and '退' not in current_data[stock].name]

# 过滤跌停的股票
def filter_limitdown_stock(stock_list):
	last_prices = jqdatasdk.history(1, unit='1m', field='close', security_list=stock_list)
	current_data = jqdatasdk.get_current_data()
	return [stock for stock in stock_list if last_prices[stock][-1] > current_data[stock].low_limit]

In [ ]:
# 过滤停牌股票
def filter_paused_stock(all,date):
    is_susp = jqdatasdk.get_price(all,end_date=date, count=1,fields='paused',panel=False)
    is_susp = is_susp[is_susp.paused==0]
    return is_susp

# 过滤ST及其他具有退市标签的股票
# 过滤ST及其他具有退市标签的股票
def filter_st_stock(stock_list,date):
    return [stock for stock in stock_list
        if 'ST' not in jqdatasdk.get_security_info(stock,date=date).display_name
        and '*' not in jqdatasdk.get_security_info(stock,date=date).display_name
        and '退' not in jqdatasdk.get_security_info(stock,date=date).display_name]